In [3]:
import cv2 
import numpy as np
net = cv2.dnn.readNet('/Users/amariofausta/Documents/Kuliah/Semester 8/Bobot/yolov3custom_last.weights copy 5','/Users/amariofausta/Downloads/yolov3custom2.cfg')
with open("/Users/amariofausta/Documents/Kuliah/Semester 8/Dataset/dataset/classes.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [4]:
import time
import sys

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
measurement_data = []
jumlah_frame = 0
detik = 0
Pembatas = []
Pembatas_bawah = 0
frame_count = 0
start_time = time.time()
catatan_waktu1 = []
catatan_waktu2 = []

# Start video capture from webcam
cap = cv2.VideoCapture('/Users/amariofausta/Documents/Kuliah/Semester 8/Video mentah/30 cm part 1.mp4')
while True:
    # Read a frame from the video capture
    ret, frame = cap.read()
    
    # Create a blob from the input image
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)
    
    # Pass the blob through the network and get the outputs
    net.setInput(blob)
    outputs = net.forward(output_layers)
    
    # Process the outputs to get the detected objects
    class_ids = []
    confidences = []
    boxes = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.4:
                # Object detected
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                width = int(detection[2] * frame.shape[1])
                height = int(detection[3] * frame.shape[0])
                x = int(center_x - width / 2)
                y = int(center_y - height / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, width, height])
        
    # Draw the detected objects on the image
    for i in range(len(boxes)):
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        measurement_data.append([y, classes[class_ids[i]]])
        cv2.putText(frame, classes[class_ids[i]], (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        #print(boxes[i],classes[class_ids[i]])

        # Mengetahui pembatas atas dan bawah
        if classes[class_ids[i]] == 'Pembatas':
            Pembatas.append(boxes[i])
        if len(Pembatas) == 2:
            if Pembatas[0][1] > Pembatas[1][1]:
                Pembatas_atas = Pembatas[1][1]
                Pembatas_bawah = Pembatas[0][1]
            elif Pembatas[0][1] < Pembatas[1][1]:
                Pembatas_atas = Pembatas[0][1]
                Pembatas_bawah = Pembatas[1][1]

        if classes[class_ids[i]] == 'Bola':
            if len(Pembatas) > 2:
                if boxes[i][1] >= Pembatas_atas and boxes[i][1] <= Pembatas_bawah:
                    catatan_waktu1.append(time.time())
                if boxes[i][1] >= Pembatas_bawah:
                    catatan_waktu2.append(time.time())

    # Display the output image
    cv2.imshow("Output", frame)
    frame_count += 1
    jumlah_frame += 1
    
    # elapsed_time = time.time() - start_time
    '''
    if elapsed_time > 1:
        fps = frame_count / elapsed_time
        #print("FPS: {:.2f}".format(fps))
        frame_count = 0
        start_time = time.time()
    '''
    # Wait for a key press to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Release resources
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
